In [322]:
pwd

'C:\\Users\\User\\Downloads\\NLP ASS 2\\NLP_Spellchecker_App-master (1)'

In [323]:
import os

In [324]:
path = "C:\\Users\\User\\Downloads\\NLP ASS 2\\NLP_Spellchecker_App-master (1)\\corpus"
dirs = os.chdir(path)

In [325]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import math
import random
import numpy as np
import pandas as pd
import nltk
import string 
import matplotlib
matplotlib.use('Agg')

In [326]:
with open("merged_corpus.txt", "r", encoding = 'ISO-8859-1') as f: 
    data = f.read()

In [327]:
#514-8 = proj gutenberg 
#medical vocab = https://github.com/socd06/medical-nlp
#medical articles = https://www.kaggle.com/trikialaaa/2k-clean-medical-articles-medicalnewstoday/tasks
#merged corpus = 514-8 + medical articles

In [328]:
#Preprocess file    
data = re.sub(r'[^A-Za-z\.\?!\']+', ' ', data) #remove special character
data = re.sub(r'\b(?:[A-Z]{3,}\b|(?:[A-Za-z]\.){2,})\s*', ' ',data)# removes 3 or more capital letters e.g. XYZ or X.Y.Z. 
sentences = re.split(r'[\.\?!]+[ \n]+', data) #split data into sentences
sentences = [s.strip() for s in sentences] #Remove leading & trailing spaces
sentences = [s for s in sentences if len(s) > 0] #Remove whitespace

In [329]:
tokenized_sentences=[]
# this is the new tokenisation for sentences. This will keep all words including "shouldn't". Removes numbers and non words like e.g. and etc. 
for sentence in sentences: 
        non_words = ['etc'] 
        # Convert to lowercase letters
        sentence = sentence.lower() 
        
        # Convert into a list of words
        tokenized = re.findall("[\w']+", sentence) #tokenises words. keeps numbers keeps shouldn't but has e and g in list from e.g. 
        
        # Remove random letters leftover from tokenising. 
        one_word = list(string.ascii_lowercase[0:26])
        one_word.remove('a')
        one_word.remove('i')
        
        for i in list(tokenized):
            if i in one_word:
                tokenized.remove(i)
                
        # remove non_words i.e. 'etc'
        for word in list(tokenized): 
                if word in non_words: 
                    tokenized.remove(word) 
        # append the list of tokenized_sentences to the list of lists
        tokenized_sentences.append(tokenized) 

In [330]:
def count_words(tokenized_sentences):
    """
    Count the number of each individual word in every sentences
    """
    word_counts = {}
    for sentence in tokenized_sentences: # complete this line
        word_counts
        # Go through each token in the sentence
        for token in sentence: # complete this line

            # If the token is not in the dictionary yet, set the count to 1
            if not token in word_counts: # complete this line
                word_counts[token] = 1
            
            # If the token is already in the dictionary, increment the count by 1
            else:
                word_counts[token] += 1

    
    return word_counts

def get_nplus_words(tokenized_sentences, n):
    """
    Find words in corpus with nplus frequency
    """
    nplus_vocab = []
    
    word_counts = count_words(tokenized_sentences)

    for word, cnt in word_counts.items():
        if cnt >= n:
            nplus_vocab.append(word)
    
    return  nplus_vocab

def replace_words_below_n_by_unk(tokenized_sentences, n=2):
    """
    Process training data to replace words with frequency less than n by UNK
    """
    vocabulary = set(get_nplus_words(tokenized_sentences, n))
    
    processed_tokenized_sentences = []
    
    for sentence in tokenized_sentences:
        temp_sentence = []
        
        for token in sentence: 
            if token in vocabulary: # complete this line
                temp_sentence.append(token)
            else:
                temp_sentence.append("<unk>")
        
        processed_tokenized_sentences.append(temp_sentence)
        
    return processed_tokenized_sentences

In [331]:
path = "C:\\Users\\User\\Downloads\\NLP ASS 2\\NLP_Spellchecker_App-master (1)"
dirs = os.chdir(path)

In [332]:
with open("medical_vocab.txt", "r", encoding = 'ISO-8859-1') as f: 
    medical_vocab = f.read()

In [333]:
medical_vocab = re.sub(r'[^A-Za-z\.\?!\']+', ' ', medical_vocab) #remove special character
medical_vocab = re.sub(r'\b(?:[A-Z]{3,}\b|(?:[A-Za-z]\.){2,})\s*', ' ',medical_vocab)# removes 3 or more capital letters e.g. XYZ or X.Y.Z. 

In [334]:
from nltk.tokenize import word_tokenize
medical_vocab = word_tokenize(medical_vocab)

In [335]:
# Get Vocabulary
vocabulary = list(set(get_nplus_words(tokenized_sentences, 2))) #words with more than 2 frequency
vocabulary = vocabulary + medical_vocab #append medical vocab to gutenberg vocab
vocabulary = vocabulary+['<s>']+['<e>']
# Replace less frequent word by <UNK>
processed_sentences =replace_words_below_n_by_unk(tokenized_sentences, 2)

In [336]:
def n_grams_dict(tokenized_sentences, n, start_token='<s>', end_token = '<e>'):
    """
    Return all n_grams with count
    """  
    n_grams = {}

    for sentence in tokenized_sentences: # complete this line
        
        # add start and end token
        sentence = n * [start_token] + sentence + [end_token]
        # convert to tuple
        sentence = tuple(sentence)

        for i in range(len(sentence)-n+1): 

            n_gram = sentence[i:i+n]
            
            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    
            ### END CODE HERE ###
    return n_grams

In [337]:
# Get the unigram and bigram
unigram_counts = n_grams_dict(processed_sentences, 1)
bigram_counts = n_grams_dict(processed_sentences, 2)

In [338]:
import numpy as np
def delete_letter(word, verbose=True):
    '''return list of word with deleted letter
    '''
    
    delete_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    delete_l = [L+R[1:] for L,R in split_l if R]

    if verbose: print(f"input word {word}, \ndelete_l = {delete_l}")

    return delete_l

def transpose_letter(word, verbose=True):
    '''
    return list of words with transposed letter
    ''' 
    
    transpose_l = []
    split_l = []

    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    transpose_l = [L + R[1] + R[0] + R[2:] for L,R in split_l if len(R)>1]
    
    if verbose: print(f"Input word = {word} \ntranspose_l = {transpose_l}") 

    return transpose_l

def subst_letter(word, verbose=True):
    '''
    return list of substitute letter
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    subst_l = []
    split_l = []

    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    subst_l = [L + l + R[1:] for L,R in split_l if R for l in letters]
    subst_set = set(subst_l)    
    subst_set.discard(word)

    # turn the set back into a list and sort it, for easier viewing
    subst_l = sorted(list(subst_set))
    
    if verbose: print(f"Input word = {word} \nsubst_l {subst_l}")   
    
    return subst_l

def ins_letter(word, verbose=True):
    '''
     return list of words with inserted letter
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    ins_l = []
    split_l = []
    

    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    ins_l = [L + l + R for L,R in split_l for l in letters]
    
    if verbose: print(f"Input word {word} \nins_l = {ins_l}")
    
    return ins_l

def edit_one_letter(word):
    """
    return a set of real/non word which is within edit distance of one 
    """
    edit_one_set = set(delete_letter(word) + transpose_letter(word) + subst_letter(word) + ins_letter(word))
    
    return edit_one_set

def edit_two_letters(word):
    '''
    return a set of real/non word which is within edit distance of two
    '''
    
    edit_one = edit_one_letter(word)
    edit_two_set = set()
    for w in edit_one:
        edit_two = edit_one_letter(w)
        edit_two_set = edit_two_set.union(edit_two_set, edit_two)
   
    return edit_two_set

def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 1, trp_cost= 1):

    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    

   # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): 
        D[row,0] = D[row-1, 0] + del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1,n+1): 
        D[0,col] = D[0, col-1] + ins_cost
                
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
              
            # Update the cost at row, col based on previous entries in the cost matrix
            D[row,col] = min (D[row, col-1] + ins_cost, D[row-1, col] + del_cost, D[row-1, col-1] + r_cost)
            # transposition  
            if row>1 and col>1 and source[row-1]==target[col-2] and source[row-2] == target[col-1]:
                D[row,col] = min (D[row,col], D[row-2,col-2] + trp_cost) 
    # Set the minimum edit distance with the cost found at row m, column n
    min_distance = D[m,n]
    
    
    return D, min_distance

In [339]:
import ast
f=open('addconfusion.data', 'r')
data=f.read()
f.close
addmatrix=ast.literal_eval(data)
f=open('subconfusion.data', 'r')
data=f.read()
f.close
submatrix=ast.literal_eval(data)
f=open('revconfusion.data', 'r')
data=f.read()
f.close
revmatrix=ast.literal_eval(data)
f=open('delconfusion.data', 'r')
data=f.read()
f.close
delmatrix=ast.literal_eval(data)

def editType(candidate, word):
        "Method to calculate edit type for single edit errors."
        edit=[False]*4
        correct=""
        error=""
        x=''
        w=''
        for i in range(min([len(word),len(candidate)])-1):
            if candidate[0:i+1] != word[0:i+1]:
                if candidate[i:] == word[i-1:]:
                    edit[1]=True
                    correct = candidate[i-1]
                    error = ''
                    x = candidate[i-2]
                    w = candidate[i-2]+candidate[i-1]
                    break
                elif candidate[i:] == word[i+1:]:
                    
                    correct = ''
                    error = word[i]
                    if i == 0:
                        w = '#'
                        x = '#'+error
                    else:
                        w=word[i-1]
                        x=word[i-1]+error
                    edit[0]=True
                    break
                if candidate[i+1:] == word[i+1:]:
                    edit[2]=True
                    correct = candidate[i]
                    error = word[i]
                    x = error
                    w = correct
                    break
                if candidate[i] == word[i+1] and candidate[i+2:]==word[i+2:]:
                    edit[3]=True
                    correct = candidate[i]+candidate[i+1]
                    error = word[i]+word[i+1]
                    x=error
                    w=correct
                    break
        candidate=candidate[::-1]
        word=word[::-1]
        for i in range(min([len(word),len(candidate)])-1):
            if candidate[0:i+1] != word[0:i+1]:
                if candidate[i:] == word[i-1:]:
                    edit[1]=True
                    correct = candidate[i-1]
                    error = ''
                    x = candidate[i-2]
                    w = candidate[i-2]+candidate[i-1]
                    break
                elif candidate[i:] == word[i+1:]:
                    
                    correct = ''
                    error = word[i]
                    if i == 0:
                        w = '#'
                        x = '#'+error
                    else:
                        w=word[i-1]
                        x=word[i-1]+error
                    edit[0]=True
                    break
                if candidate[i+1:] == word[i+1:]:
                    edit[2]=True
                    correct = candidate[i]
                    error = word[i]
                    x = error
                    w = correct
                    break
                if candidate[i] == word[i+1] and candidate[i+2:]==word[i+2:]:
                    edit[3]=True
                    correct = candidate[i]+candidate[i+1]
                    error = word[i]+word[i+1]
                    x=error
                    w=correct
                    break
        if word == candidate:
            return "None", '', '', '', ''
        if edit[1]:
            return "Deletion", correct, error, x, w
        elif edit[0]:
            return "Insertion", correct, error, x, w
        elif edit[2]:
            return "Substitution", correct, error, x, w
        elif edit[3]:
            return "Reversal", correct, error, x, w
        
def channelModel(x,y, edit,corpus):
        """Method to calculate channel model probability for errors."""
        
        if edit == 'add':
            if x == '#':
                return addmatrix[x+y]/corpus.count(' '+y)
            else:
                return addmatrix[x+y]/corpus.count(x)
        if edit == 'sub':
            return submatrix[(x+y)[0:2]]/corpus.count(y)
        if edit == 'rev':
            return revmatrix[x+y]/corpus.count(x+y)
        if edit == 'del':
            return delmatrix[x+y]/corpus.count(x+y)

In [340]:
def get_probability(previous_n_words, word, 
                         previous_n_gram_dict, n_gram_dict, vocabulary_size, k=1.0):
    """
    Return N-gram probability given the pair of current word and previous_n_words
    """
    assert type(previous_n_words) == list
    # convert list to tuple to use it as a dictionary key
    previous_n_words = tuple(previous_n_words,)
    
    previous_n_words_count = previous_n_gram_dict[previous_n_words] if previous_n_words in previous_n_gram_dict else 0

    # k-smoothing
    denominator = previous_n_words_count + k*vocabulary_size

    # Define n plus 1 gram as the previous n-gram plus the current word as a tuple
    n_gram = previous_n_words + (word,)

    n_gram_count = n_gram_dict[n_gram] if n_gram in n_gram_dict else 0
   
    # smoothing
    numerator = n_gram_count + 1

    probability = numerator/denominator
    
    
    return probability

In [341]:
def get_corrections(previous_n_words_i, word, vocab, n=2, verbose = False):
    '''
    Get n candidates with individual probability
    '''
    assert type(previous_n_words_i) == list
    corpus = ' '.join(vocabulary)
    suggestions = []
    n_best = []
    ### Convert to UNK if word not in vocab
    previous_n_words = []
    for w in previous_n_words_i:
        if w not in vocabulary:
            previous_n_words.append('<unk>')
        else:
            previous_n_words.append(w)
            
    ##Suggestions include input word only if the input word in vocab
    if word in vocab:    
        suggestions = [word] + list(edit_one_letter(word).intersection(vocabulary)) or list(edit_two_letters(word).intersection(vocabulary)) 
    else:
        suggestions = list(edit_one_letter(word).intersection(vocabulary)) or list(edit_two_letters(word).intersection(vocabulary)) 
        
    words_prob = {}
    for w in suggestions: 
        # To make sure all suggestions is within edit distance of 2
        min_edits = min_edit_distance(word,w)[1]
        if not word in vocab: ##use error model only when it is non word error
            if min_edits <= 2:
                edit = editType(w,word)
                if edit[0] == "Insertion":
                    error_prob = channelModel(edit[3][0],edit[3][1], 'add',corpus)
                if edit[0] == 'Deletion':
                    error_prob = channelModel(edit[4][0], edit[4][1], 'del',corpus)
                if edit[0] == 'Reversal':
                    error_prob = channelModel(edit[4][0], edit[4][1], 'rev',corpus)
                if edit[0] == 'Substitution':
                    error_prob = channelModel(edit[3], edit[4], 'sub',corpus)
            else:
                error_prob = 1
                print('error from here')
        else:
            error_prob = 1
            print('no error is from here')
        language_prob = get_probability(previous_n_words, w, 
                        unigram_counts, bigram_counts, len(vocabulary), k=1.0)
            

        words_prob[w] = language_prob * error_prob
        print('words_prob = ', words_prob)
        print('error_prob =',error_prob)
        print('language_prob = ', language_prob)
    n_best = Counter(words_prob).most_common(n)
    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best


In [342]:
#GUI Creations

In [343]:
# GUI CREATION THROUGH PYTHON'S TKINTER LIBRARY
from tkinter import *

# creates a base GUI window
root = Tk() 

# creating fixed geometry of the tkinter window with dimensions 700x900
root.geometry("700x900") 
root.configure(background = "light blue")

root.title("NLP Spell Checker") # Adding a title to the GUI window.
Label(root, text = "NLP Assignment 2", fg = "navy", bg = "gray", font = "Arial 11 bold italic", height = 3, width = 200).pack()

In [344]:
# function to retrieve the sentence typed by a user & pass the input through get_corrections() to check spellings
tokenized_sentence = []
non_real_word = []

clicked=StringVar()

In [345]:
def getInput():
    global tokenized_sentence
    # Preprocess the original text input to get clean input
    sentenceValues = enteredSentence.get('1.0', '50.0') #get input sentence
    sentenceValues = sentenceValues.lower()
    outputSentence.delete(0.0, 'end')
    outputSentence.insert(END, sentenceValues)  
    
    # tokenize the sentence and save the values to tokenizedWords variable
    tokenized_sentence = nltk.word_tokenize(sentenceValues)
    tokenized_sentence = ['<s>']+ tokenized_sentence
    
    not_in_corpus=[]
    real_word_error=[]
    for word in tokenized_sentence:
        if word not in vocabulary:
            not_in_corpus.append(word)  # Saving non real word to not_in_corpus list.

    for word in tokenized_sentence[1:]:
        if word in vocabulary: 
            index=tokenized_sentence.index(word)
            candidate_words = get_corrections([tokenized_sentence[index-1]], word, vocabulary, n=1, verbose=True)
            if candidate_words[0][0] != word :
                real_word_error.append(word) # saving a real & existing word to real_word_error
    print(real_word_error)            
    print("Suitable candidate words are:")
    
    # Checking for non_word errors from the input sentence typed by a user
    options=[]
    for word in not_in_corpus:
        
        offset = '+%dc' % len(word) # +5c (5 chars)
    
        # search word from first char (1.0) to the end of text (END)
        pos_start = enteredSentence.search(word, '1.0', END)
        
        # check if the word has been found
        while pos_start:
        
            # create end position by adding (as string "+5c") number of chars in searched word 
            pos_end = pos_start + offset
        
            # add tag
            enteredSentence.tag_add('red_tag', pos_start, pos_end)
        
            # search again from pos_end to the end of text (END)
            pos_start = enteredSentence.search(word, pos_end, END)
        
        options.append(word)   
    
    # checking for real word error from the input sentence by a user
    for word in real_word_error:
        offset = '+%dc' % len(word) # +5c (5 chars)
    
        # search word from first char (1.0) to the end of text (END)
        pos_start = enteredSentence.search(word, '1.0', END)
        
        # check if the word has been found
        while pos_start:
        
            # create end position by adding (as string "+5c") number of chars in searched word 
            pos_end = pos_start + offset
        
            # add tag
            enteredSentence.tag_add('blue_tag', pos_start, pos_end)
        
            # search again from pos_end to the end of text (END)
            pos_start = enteredSentence.search(word, pos_end, END)
        
        options.append(word)   
    
    # Creating a drop down menu to display the misspelled words.
    # From this drop down list, a user selects the misspelled word that they need suggestions for.
    drop = OptionMenu(root,clicked,*options)
    drop.configure(font=("Arial", 10))
    drop.pack()
    drop.place(x=305, y = 350)


In [346]:
# Function to display a list of the suggested words
def showSuggestions():
    suggestedWords.delete(0, END)
    options=[]
    word_to_replace = clicked.get()
    index=tokenized_sentence.index(word_to_replace)
    
    candidate_words = get_corrections([tokenized_sentence[index-1]], word_to_replace, vocabulary, n=3, verbose=False)
    print(candidate_words)
    for i in range(len(candidate_words)):
        suggestedWords.insert(END,candidate_words[i][0])


In [347]:
# Function to replace a misspelled word with the correct word from a list of suggested words            
def replace_word():
    word_to_replace = clicked.get()
    selected_word=suggestedWords.get(ANCHOR)
    offset = '+%dc' % len(word_to_replace) # +5c (5 chars)
    idx = '1.0'
    #searches for desried string from index 1  
    idx = outputSentence.search(word_to_replace, idx, nocase = 1,  
                            stopindex = END) 
    # last index sum of current index and  
    # length of text  
    lastidx = '% s+% dc' % (idx, len(word_to_replace)) 
  
    outputSentence.delete(idx, lastidx) 
    outputSentence.insert(idx, selected_word) 
  
    lastidx = '% s+% dc' % (idx, len(selected_word))  
            

In [348]:
# Function to highlight the listbox from keyword search 
def highlight():
    searchx = SEARCH.get()
    searchx = searchx.lower()
    for i,item in enumerate (all_listbox_item):
        if searchx in item:
            listbox.selection_set(i) #highlights word
            listbox.see(i) #scrolls down to word if not within the listbox
        else:
            listbox.selection_clear(i)
        if searchx == '':
            listbox.selection_clear(0,"end")

In [349]:
# Function to clear the text in Searchbox    
def Reset():
    global SEARCH
    clearinput = ""
    SEARCH.set(clearinput)

In [350]:
# Function to enable a pop up window on right click
def do_popup(event): 
    try:
        m.tk_popup(event.x_root, event.y_root)
    finally:
        m.grab_release()    

# when the popup window opens on right click, it will have a word called suggest which will execute the showSuggestions function
m = Menu(root, tearoff = 0) 
m.add_command(label ="suggest", command=showSuggestions) 

In [ ]:
SEARCH = StringVar()
# Input widget for sentence to be entered by user
Label(text="Enter sentence here (Max Words: 500)", font="Arial 11 bold").place(x=15, y=80)
enteredSentence = Text(root, height = 10, width = 60)
enteredSentence.configure(font=("Arial", 11))
enteredSentence.place(x=15, y=110)
submit_btn = Button(root, height=1, width=10, text="Submit", command=getInput).place(x=585, y=110)
enteredSentence.tag_config("red_tag", foreground="red", underline=1)
enteredSentence.tag_config("blue_tag", foreground="blue", underline=1)
enteredSentence.tag_bind("red_tag", '<Button-3>', do_popup) #calls the pop up funtion
enteredSentence.tag_bind("blue_tag", '<Button-3>', do_popup) #calls the pop up funtion

# Creating a suggestions widget for the suggested words to correct the mispelled word
Label(text="List of suggested words to replace misspelled word:", font = "Arial 11 bold").place(x=15, y=320)
suggestedWords = Listbox(root, height = 10, width = 30)
suggestedWords.configure(font=("Arial", 11))
#suggestedWords.config(state = "disabled")
suggestedWords.place(x=15, y = 350)
sugg_btn = Button(root, text="Show suggestions", command=showSuggestions).place(x=305, y=380)
replace_btn = Button(root, text="Replace Word", command=replace_word).place(x=305, y=410)

# Output widget for the sentence entered and open for correcting mispelled words
Label(text="Corrected Input Sentence by User:", font = "Arial 11 bold").place(x=15, y=560)
outputSentence = Text(root, height = 10, width = 60, wrap=WORD)
outputSentence.configure(font=("Arial", 11))
#outputSentence.config(state = "disabled")
outputSentence.place(x=15, y=590)

#---------------------------------WIDGETS FOR DICTIONARY-----------------------------------

#=====================================LABEL WIDGET=========================================
lbl_title = Label(root, width=20, font=('arial', 8), text="Dictionary").place(x=550, y=360) 

#=====================================ENTRY WIDGET=========================================
search_entry = Entry(root, textvariable=SEARCH).place(x=550, y=530)

#=====================================BUTTON WIDGET========================================
btn_search = Button(root, text="Search", bg="#006dcc", command=highlight).place(x=550, y=555) 
btn_reset = Button(root=search_entry, text="Reset", bg="#ff0000",command=Reset).place(x=635, y=555) 

#=====================================Table WIDGET=========================================
#highlight the selection in listbox
listbox = Listbox(root, height=9)
selection = listbox.curselection()
listbox.place(x=550,y=380) 

#upload txt to listbox and populate it
f = vocabulary

for x in f:
    listbox.insert("end", x)

all_listbox_item = listbox.get(0, "end")


# Activating the GUI
root.mainloop()

[]
Suitable candidate words are:
input word kidny, 
delete_l = ['idny', 'kdny', 'kiny', 'kidy', 'kidn']
Input word = kidny 
transpose_l = ['ikdny', 'kdiny', 'kindy', 'kidyn']
Input word = kidny 
subst_l ['aidny', 'bidny', 'cidny', 'didny', 'eidny', 'fidny', 'gidny', 'hidny', 'iidny', 'jidny', 'kadny', 'kbdny', 'kcdny', 'kddny', 'kedny', 'kfdny', 'kgdny', 'khdny', 'kiany', 'kibny', 'kicny', 'kiday', 'kidby', 'kidcy', 'kiddy', 'kidey', 'kidfy', 'kidgy', 'kidhy', 'kidiy', 'kidjy', 'kidky', 'kidly', 'kidmy', 'kidna', 'kidnb', 'kidnc', 'kidnd', 'kidne', 'kidnf', 'kidng', 'kidnh', 'kidni', 'kidnj', 'kidnk', 'kidnl', 'kidnm', 'kidnn', 'kidno', 'kidnp', 'kidnq', 'kidnr', 'kidns', 'kidnt', 'kidnu', 'kidnv', 'kidnw', 'kidnx', 'kidnz', 'kidoy', 'kidpy', 'kidqy', 'kidry', 'kidsy', 'kidty', 'kiduy', 'kidvy', 'kidwy', 'kidxy', 'kidyy', 'kidzy', 'kieny', 'kifny', 'kigny', 'kihny', 'kiiny', 'kijny', 'kikny', 'kilny', 'kimny', 'kinny', 'kiony', 'kipny', 'kiqny', 'kirny', 'kisny', 'kitny', 'kiuny', 'kiv